In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Instructions for updating:
non-resource variables are not supported in the long term
Num GPUs Available:  1


In [2]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

#params['dataset'] = 'sent140'
params['dataset'] = 'mnist'
params['model'] = 'mlp'
#params['model'] = 'stacked_lstm'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'].startswith('mnist'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
  params['num_group'] = 3
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
  params['num_group'] = 5
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
  params['num_group'] = 5
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 5

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['proximal'] = False
  params['RAC'] = False
  params['RCC'] = False
  params['allow_empty'] = True
  
  # TEST Hyperparameters, Experimental Only
  params['evenly'] = False
  params['min_clients'] = 2
    
  # We implement IFCA and FeSEM base on grouprox
  params['ifca'] = False
  params['fesem'] = False
    
"""
Common parameters
"""
params['num_rounds'] = 300
params['num_epochs'] = 20
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20 ################ Important ######################
params['seed'] = 233
params['agg_lr'] = 0.05

"""
We immplement IFCA and FeSEM base on grouprox,
Some configurations should be flush for above two CFL frameworks
"""
if params['ifca'] == True or params['fesem'] == True:
    params['proximal'] = False
    params['RCC'] = False
    params['allow_empty'] = True
    params['agg_lr'] = 0 # IFCA and FeSEM don't use this parameter

""" Set the output CSV file name """
info = '{}-{}-{}'.format(params['optimizer'], params['dataset'], params['model'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
    
  if params['ifca'] == True:
    info2 = info2 + '-IFCA'
  if params['fesem'] == True:
    info2 = info2 + '-FeSEM'
  
  info2 = '-NumG{:1d}-Agglr{:.4}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['proximal']))
  
  if params['RCC'] == True:
    info2 = info2 + '-RCC.{:.1}'.format(str(params['RCC']))
  
  if params['RAC'] == True:
    info2 = info2 + '-RAC.{:.1}-Evenly{:.1}'.format(str(params['RAC']), str(params['evenly']))
  elif params['allow_empty'] == True:
    info2 = info2 + '-Empty{:.1}'.format(str(params['allow_empty']))
  elif params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

grouprox-mnist-mlp-NumG3-Agglr0.05-ProximalF-EmptyT.csv


In [3]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
elif params['dataset'].startswith('mnist'):
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'mnist', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])
mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
params['sklearn_seed'] = params['seed']
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)  
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Group prox to Train


/home/lab/anaconda3/envs/FL/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/lab/anaconda3/envs/FL/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch:   0%|                                                                                      | 0/1 [00:00<?, ?it/s]

1000 Clients in Total
101770 Parameters in Total


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Pre-training takes 1.6543328762054443s seconds
SVD takes 0.14454436302185059s seconds
Delta w shape: (60, 101770) Matrix V shape: (101770, 3)
Ternary Cossim Matrix calculation takes 0.01588582992553711s seconds
Clustering takes 0.018156051635742188s seconds
Clustering Results: Counter({1: 31, 2: 16, 0: 13})
Clustering Inertia: 1.0033761262893677


In [4]:
"""
Test Ternary Similarity
"""
#x, y = server.test_ternary_cosine_similariy()

'\nTest Ternary Similarity\n'

In [5]:
"""
import matplotlib.pyplot as plt
plt.figure(figsize=(7,7))
plt.scatter(x[:50000], y[:50000], linewidths=0.01, alpha=0.2, norm=True)
plt.show()
"""

'\nimport matplotlib.pyplot as plt\nplt.figure(figsize=(7,7))\nplt.scatter(x[:50000], y[:50000], linewidths=0.01, alpha=0.2, norm=True)\nplt.show()\n'

In [ ]:
server.train()

Training with 20 workers ---


Group 0, clients ['f_00859', 'f_00298', 'f_00558', 'f_00239', 'f_00578']
Group 1, clients ['f_00993', 'f_00027', 'f_00306', 'f_00706', 'f_00496', 'f_00784', 'f_00055', 'f_00906', 'f_00175', 'f_00014', 'f_00077']
Group 2, clients ['f_00553', 'f_00672', 'f_00971', 'f_00231']


Epoch:  45%|██████████████████████████████████▋                                          | 9/20 [00:00<00:00, 83.66it/s]

Group 0
At round 0 accuracy: 0.6532929598788796
At round 0 training accuracy: 0.6520510483135825
At round 0 training loss: 1.2432568036216765
Group 1
At round 0 accuracy: 0.3258590564938847
At round 0 training accuracy: 0.3295271453590193
At round 0 training loss: 2.0476418802408327
Group 2
At round 0 accuracy: 0.5764525993883792
At round 0 training accuracy: 0.5600162572254336
At round 0 training loss: 1.351473760628276
At round 0 mean test accuracy: 0.4734771401438068 mean train accuracy: 0.46967436429683446
number: [5, 11, 4]
The client-group discrepancy are: [0.32899693 0.43719464 0.25975225 0.38417265]
Begin group  0 training


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

...

Begin group  2 training


Group 0, clients ['f_00609', 'f_00449', 'f_00908', 'f_00618', 'f_00439', 'f_00289']
Group 1, clients ['f_00414', 'f_00707', 'f_00855', 'f_00776', 'f_00253', 'f_00087', 'f_00186']
Group 2, clients ['f_00662', 'f_00280', 'f_00110', 'f_00681', 'f_00742', 'f_00840', 'f_00573']


Epoch:  35%|██████████████████████████▉                                                  | 7/20 [00:00<00:00, 69.17it/s]

Group 0
At round 65 accuracy: 0.967448902346707
At round 65 training accuracy: 0.9767547857793983
At round 65 training loss: 0.07288226219338018
Group 1
At round 65 accuracy: 0.9274898078043099
At round 65 training accuracy: 0.9359019264448336
At round 65 training loss: 0.20570130933991154
Group 2
At round 65 accuracy: 0.9701834862385321
At round 65 training accuracy: 0.9771044075144508
At round 65 training loss: 0.09302485468438293
At round 65 mean test accuracy: 0.9498032831366164 mean train accuracy: 0.9579657498702646
number: [6, 7, 7]
The client-group discrepancy are: [2.02643267 1.7061107  2.71214383 1.61528321]
Begin group  0 training


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

In [ ]:
"""
fname = 'sampling_interval.csv'
interval = 5
rounds = 200
with open(fname, 'w') as csv_f:
  csv_f.write(str(1)+'\n')
  for i in range(2, rounds+1):
    if i % interval == 0:
      csv_f.write(str(i))
    elif i == rounds:
      csv_f.write('200')
    csv_f.write('\n')
"""